# 闭包（closure）
- 闭包是由函数及相关的引用环境组合而成的实体
- 当一个函数在内部定义函数，并且内部的函数使用外部函数的参数或者是局部变量，当内部函数被当做返回值的时候，相关函数和变量保存在返回的函数中，这种结果就叫闭包（如果在一个函数中，定义了另外一个函数，并且那个函数使用了外面函数的变量，并且外面那个函数返回了里面这个函数的引用 ，那么称里面的这个函数为闭包）
    - 归根结底：闭包要满足以下两点要求
        - （1）必须要有函数的嵌套。而且外层函数必须返回内层函数，但是内层函数可以不返回值，也可以返回值；外层函数给内层函数提供了一个“包装起来的运行环境”，在这个“包装的”运行环境里面，内层函数可以完全自己做主。这也是为什么称之为闭包的原因了。
        - （2）内层函数一定要用到外层函数中定义的变量。如果只满足了特征（1），也不算是闭包，一定要用到外层“包装函数”的变量，这些变量称之为“自由变量”。
        

###  nonlocal关键字
- 如果想要在闭包中修改外面函数的变量，这时候应该使用nonlocal关键字，来吧这个变量标识为外面函数的变量（即非局部变量）

- f.__code__.co_freevars:查看函数b的自由变量的具体值，而f.__closure__返回一个自由变量组成的元组，通过f.__closure__[i].cell_contents 查看第几个自由变量的值
    - 自由变量：
        - 在闭包中，假如有一个参数a为外部函数的局部变量，其被分配的内存在外部函数执行后应该被立即释放，但在外部函数执行后，发现自己的局部变量将被内部函数调用，就把这个变量绑定给了内部函数，然后自己再结束。此时的局部变量a就被成为自由变量
- b.__code__.co_varnames:查看函数b的局部变量
- 闭包会坑人,所以返回函数不要引用任何循环变量，或者后续会发生变化的变量。

In [ ]:
# 闭包的常见坑之一
"""
def fw(a, b):
    def area(x):
        a += 1
        return a * b -x
    return area
a = fw(2,3)
print(a(4))
"""
# 以上函数直接运行会报错，原因是在area函数中a += 1等价于a = a+1,这句话表明在内部函数area创建了新的局部变量a,
# 而在area函数中并没有声明局部变量a,找不到就会报错。而由于新的局部变量的创建，就不会引用外部的自由变量a,也就是说，
# 此时的变量a并不是外部函数中的变量，而闭包也不存在了。解决办法如下
# 1.将area中的变量声明为非局部变量，即：nonlocal a
# 2.更改变量名,使得新建立的变量与外部函数的变量名即a不同

"""
def fw(a, b):
    def area(x):
        c = a + 1
        return c * b -x
    return area
"""
def fw(a, b):
    def area(x):
        nonlocal a
        a += 1
        return a * b -x
    return area
a = fw(2,3)
print(a(4))
#print(locals())

In [ ]:
# 闭包常见坑之二
def count():
    fgs = []
    for i in range(4):
        def f():
            return i*i
        fgs.append(f)
    print(1+i)
    return fgs
f1,f2,f3,f4 = count()
print(f1(),f2(),f3(),f4())
# 此结果与预想不一样


# 坑的解释
- 造成上述问题的原因是，返回函数引用了变量， i并非立即执行，而是等到三个函数都返回的时候才统一调用，此时i已经变成了3，最终调用的时候，返回的都是3*3
                这是因为当把函数加入fgs列表里时，python还没有给i赋值，只有当执行时，再去找i的值是什么，这时在第一个for循环结束以后，i的值是3，所以以上代码的执行结果是9,9,9,9
- 铁律：返回闭包时，返回函数不能引用任何循环变量;闭包中是不能修改外部函数中的局部变量的
- 解决办法：再创建一个函数，用该函数的参数绑定循环变量的当前值，无论该循环变量如何改变，已经绑定的函数参数值不再改变
    - 参看以下案例，本质上第二种办法把i赋值给y就是用当前函数的参数绑定循环变量的当前值

In [1]:
# 修改上述函数
# 第一种方法
def new_count():
    def f(j):
        def g():
            return j*j
        return g
    fgs = []
    for i in range(0,4):
        fgs.append(f(i))
    return fgs
f1,f2,f3,f4 = new_count()
print(f1(),f2(),f3(),f4())
print(f1.__closure__)
print(f1.__code__.co_freevars)
print("++++++分隔符+++++++++")

#第二种方法:申明这种方法是自己想出来的
def count1():
    fgs = []
    for i in range(4):
        # 把y=i移到这里也没用，没起到绑定的作用
        def f(y=i):
            return y*y
        fgs.append(f)

    return fgs
f1,f2,f3,f4 = count1()

print(f1(),f2(),f3(),f4())
print(f1.__closure__)
print(f1.__code__.co_freevars)

0 1 4 9
(<cell at 0x000001C21CF48B28: int object at 0x00007FFEA6CF9320>,)
('j',)
++++++分隔符+++++++++
0 1 4 9
None
()


In [4]:
def suibian():
    vacant = []
    for i in range(3):
        def suisui(x=3):
            return i+x
        vacant.append(suisui)
    return vacant
a = suibian()
f1, f2 , f3 = a
print(f1(), f2(), f3())

# 注意观察改变
def suibian1():
    vacant = []
    for i in range(3):
        def suisui(x=3, y=i):
            return y+x
        vacant.append(suisui)
    return vacant
b = suibian1()
f1, f2 , f3 = b
print(f1(), f2(), f3())
for j in b:
    print(j.__code__.co_varnames)

5 5 5
3 4 5
('x', 'y')
('x', 'y')
('x', 'y')


In [3]:
def createCounter():
    a = 0
    def counter():
        nonlocal a
        a += 1
        return a
    return counter

counterA = createCounter()
print(counterA(), counterA(), counterA(), counterA(), counterA()) # 1 2 3 4 5
counterB = createCounter()
if [counterB(), counterB(), counterB(), counterB()] == [1, 2, 3, 4]:
    print('测试通过!')
else:
    print('测试失败!')

1 2 3 4 5
测试通过!


In [ ]:
def is_odd(num=2):
    return lambda x: x % num == 0
L = list(filter(is_odd, range(1, 20)))

# 闭包和装饰器的比较
### 相同点：
        （1）都是函数的嵌套，分为外层函数和内层函数，而且外层函数要返回内层函数

        （2）代码的实现逻辑大同小异

        （3）二者都可以实现增加额外功能的目的——比如上面的“加法加密运算”
### 不同点：
        （1）外层函数不同，装饰器的外层函数称之为decorator，闭包的外层函数称之为闭包函数closure

        （2）外层函数的目的不同，装饰器的外层函数主要是提供函数形参function，闭包的形参主要目的是提供自由变量。

        （3）二者的特征不一样。装饰器的外层函数可以不提供自由变量，但是闭包的的外层函数一定要提供自由变量，因为如果不提供自由变量，闭包的存在就毫无意义了，即内层函数所依赖的变量却在闭包中根本没有，那还要闭包干什么？

        （4）二者的主要目的不同。装饰器的目的：代码重用+额外功能

        闭包的主要目的：保存函数的运行环境+保存闭包的局部变量。虽然二者可以有一些交集。

        （5）闭包和装饰器本质上还是不一样的，但是从形式上来说，大致可以认为闭包是装饰器的子集。记住：仅仅是从形式上哦！

# 装饰器 （decrator）
- 装饰器：提供了一种在不需要修改原函数的条件下使用其他函数修改函数的方法，本质上，就是一个返回函数的高阶函数
            说白了，就是函数嵌套，类似闭包
- 装饰器原理
        装饰器将一个完整的事情分成两部分，一部分是我们常见的，即hello（）部分，另外一部分完成一些辅助功能，即__decorator()部分。需要注意的是我们直接调用的不是hello()部分，而是__decorator()部分，而它包含hello()部分，以下是装饰的工作原理
        为了简化，可以使用 @ 将这两部分功能连接起来
- 使用：使用@语法，即每次扩展到函数定义前使用@+函数名
- 好处：一经定义，则可以装饰任意函数
- 装饰器装饰带有参数的函数时，需要通过装饰函数将参数传递给被修饰函数，详情见下面示例
- 装饰器也可以自己带参数，当然它有一个固定的参数即被修饰的函数
    - 添加方法：在装饰器外面再给他添加一个外层装饰器，该装饰器带的参数就可以传递给内层装饰器了
    
### functools.warps(func):
- 在python中一切皆对象，它们都有一个属性__name__,可以查看它的名字
- 需要导入functools模块
- 在装饰器中，需要把被装饰函数的属性复制到wrapper()函数中，否则，有些依赖函数签名的代码执行就会出错。而functools.wraps(func)就是干这个的，把它放在包装真正执行函数的函数（比如wrapper）之前就行 
    
### 装饰器类型
- 函数装饰器装饰函数
- 函数装饰器装饰类
- 类装饰器装饰函数
- 类装饰器装饰类

In [5]:
# 装饰器的原型
"""
    装饰器将一个完整的事情分成两部分，一部分是我们常见的，即hello（）部分，
    另外一部分完成一些辅助功能，即__decorator()部分。需要注意的是我们直接调用的不是hello()部分，
    而是__decorator()部分，而它包含hello()部分，以下是装饰的工作原理
    
    为了简化，可以使用 @ 将这两部分功能连接起来
"""
def indirect_call_func(func):
    def __decorator():
        # func.__name__得到func的名字
        print("进入装饰器装饰{}".format(func.__name__))
        func()
        print("退出装饰器")
    return __decorator

def hello():
    print("hello1 world!")

# 第二种调用装饰器的方法，对原始方法进行简化，使用@
@indirect_call_func
def hello2():
    print("hello2 world, nice to meet you ")

# 运行装饰器
# 第一种使用装饰器的方法，原始方法，也就是装饰器的原理
decorator_func = indirect_call_func(hello)
decorator_func()

hello2()

进入装饰器装饰hello
hello1 world!
退出装饰器
进入装饰器装饰hello2
hello2 world, nice to meet you 
退出装饰器


In [20]:
# functools.wraps原理
import functools
import time

def decorator(func):
    def wrapper(*args, **kwargs):
        print("函数开始执行")
        func(*args, **kwargs)
        print("函数 结束执行")
        return None
    return wrapper

def add(x, y):
    print("%s + %s = %s"%(x, y, x+y))
    
add(2, 3)
print(add.__name__)  # 查看该函数的签名
print("=" * 30)

@decorator
def add(x, y):
    print("%s + %s = %s"%(x, y, x+y))
print(add.__name__)
print("=" * 30)
# 一旦使用装饰器，其签名就会发生改变
# functools就是来解决这个问题的

# 上面使用装饰器的代码就相当于：
add = decorator(add)
print(add(2, 4))
print(add.__name__)

2 + 3 = 5
add
wrapper
函数开始执行
函数开始执行
2 + 4 = 6
函数 结束执行
函数 结束执行
None
wrapper


In [ ]:
# functools.wraps的使用
def print_time(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        start = time.perf_counter_ns()
        #func(*args, **kwargs)
        end = time.perf_counter_ns()
        print(func.__name__)
        print("该程序所花费时间为{:.2f} ns".format(end - start))
        return func(*args, **kwargs)
    return wrapper

@print_time
def hello(a, b):
    print(f"hello every, my value is {a+b}")
    return None
    
print(hello(1, 4))